I’ve got an assertion to make. Let me give it a try:

*“It’s difficult to write bug-free machine learning code.”*

That’s a bit of a platitude. There are plenty of bugs one could encounter in ML code: misaligned tensors, floating point error explosion, improper dataset segmentation… the list goes on. None of these seem to be a uniquely difficult class of bug - harder to find and fix than those we might find in a program like a webserver or syntax parser. Even the tricky problems around parallel processing or offloading computation to GPUs aren’t much harder than what you can find with device drivers and multithreading elsewhere. There is the argument that ML has some very difficult algorithms to understand correctly, but I’ll throw that in the bin of “Computer Science” and ignore it for this analysis. So what is my assertion about - giving me the benefit of the doubt here, and assuming that I actually have something worthwhile to say?

Another whack at stating my little thesis: 

*“The failure modes of machine learning make bug free code difficult to write.”*

Okay, this is a bit better, but vague. What are the failure modes of machine learning programs? 
In particular, we want to know what the unique failure modes are. 
At one of the spectrum, we have coding problems like tensor misalignment, or filling up our device memory and getting a driver error. 
As we agreed earlier, these aren’t meaningfully unique to machine learning programs. And in fact, the failure modes for these problems are pretty good, in the sense that they help us fix the bugs. When a tensor is misaligned, we get an exception and the program stops. We can debug the cause and fix it. When the GPU starts filling up, we can analyze how much data we’re putting on it, and try to lower our batch size or unrolled model size. 

We also have problems like statistical bias, and poorly selected or optimized algorithms that take too long to run. These sometimes have good failure cases - if our algorithm is slow, we can analyze it with profiling tools, look at the literature for more appropriate alternative algorithms, and so on. Statistical bias is harder - in some cases, we can examine, manually or statistically, the outputs of our models for characteristic bias. In other cases, the possibility of indiscernible bias is arguably a fundamental limitation for the appropriate applications of machine learning. However, I’m also going to throw this problem, for the course of this article, in the bucket labeled “~~Computer Science~~Computer Science and Statistics.” These aren’t coding problems, and not what I’m trying to assert something about right now. 

What is left are the classes of bug where we do not have a dramatic software failure mode like our code crashing or the model failing to descend - and we also assume that we do not have “bugs” in our assumptions about the model we have decided to implement. In other words, we have not set down a flawed model (like a neural network with no activations) on paper before beginning to implement it. What are these failure modes? 
They are the cases where the model is specified correctly, our implementation runs, it completes the task to some degree of performance, and yet has something wrong with it. 

For instance, it might be the case that the model is a neural network, and is missing an activation - though we have it on the diagram of the model we drew before coding. The layers sandwiching the “missing” activation can only learn a linear transform. What happens in this case? Nothing dramatic, often. The model descends as usual, and perhaps converges. The performance, however, is potentially lower than if our specified model was implemented correctly.
I claim that this kind of failure is somewhat unique to machine learning code. 

The failure mode is “soft.” The bug does not make itself known. The cost of the bug is quantitative, not qualitative. There are other quantitative bugs. Performance bugs are far more accessible to analysis, such as through profiling and looking for hot spots. Bugs in floating point and algebra libraries are similar, but the output states of these programs are well-defined. 

The notion of a model implementation’s correctness is not well defined. Often, it is completely ad-hoc, and exists only in the author’s mind. And it is hard to analyze the loss performance of most machine learning models in the way that we can analyze the hot spots and other speed or memory performance characteristics of programs themselves. There are ways of doing this - looking at learned activations and descent profiles and so on - but they are not formalized or common. 

I argue that there are two dynamics at work that interact to cause these soft failure modes. The first is the nature of gradient-based optimization, which is the dominant paradigm for ML model training (proponents of GA and similar techniques not withstanding). We can formalize a gradient-descended model as a tuple  $<W, L, G>$  (weights $W$, loss function $L:W \to Z$, gradient function $G:Z \to  L$). If everything is specified and implemented correctly, then we expect that $L(W) < L(W+\lambda (G(L(W)))$, for $\lambda$ very small. This is simply the definition of the gradient. 

What if the elements don’t do exactly what we want them to? Specifically, what if the gradient is computed incorrectly? We now have a tuple $<W, L, G’>$, where $G \neq G’$. We have no relationship between $L(W)$ and $L(W+\lambda (G(L(W)))$. GD shouldn’t do anything at all - just a random walk.

But what if $G$ is correlated with $G’$? It seems reasonable that GD might do something useful here, and indeed it does. This is the concept behind stochastic gradient descent, of course: approximate the true gradient with a minibatch gradient, and descend as if it was the real one. Here, we actually have a different tuple: $<W, L’, G>$. $L’$ is the loss function taken over the batch, not the sample population. The gradient, presumably, is correct, but taken over the wrong points. We can recover the second tuple $<W, L, G’>$ by defining $G’ = G(L’(W))$ - now we approximate the true loss with a different gradient, which we assert is correlated with the true gradient. Note that we can never actually compute the true loss L for most problems.  

Let’s assume, boldly, that this result generalizes to all tuples $<W, L, G’>$. We can descend, with some degree of effectiveness, with a gradient that is merely correlated with the true gradient. This means that if our gradient function is implemented incorrectly, but the function it implements is correlated with the gradient, the model will descend and possibly converge! The failure mode is soft. If we don’t know what our model should be able to achieve in terms of loss, we may never know that this bug exists.

Today, with a few battle-tested autograd implementations, it may be quite unlikely for us to encounter bugs in our gradient. The loss function is easier to err in, especially with complex tasks that don't fit easily into the common classification/regression/reconstruction use cases. However, there is another way that SGD can have a "soft" failure. Let’s replace the model in our tuple with $W’$ - an incorrectly implemented model. This could be the model with a missing activation we discussed earlier. Again, we will often find our model descending and converging, through possibly above the loss a correctly specified model could achieve. In this case, however, we are not encountering a problem with SGD.

Consider a toy problem that consists of computing an XOR and performing a linear regression. The loss is a weighted sum of the two task-specific losses. We will model the problem with a neural network that contains one hidden layer, using a nonlinear activation. When we forget the activation, our model is linearized, and cannot learn an XOR, but still descends and minimizes one component of the loss. When we correctly implement the specified model, the loss after training will be lower than that for the incorrect model. However, there was never any indication during training the model without an activiation that it was not correct. The model failed silently. If this was not a toy problem, and the loss was more opaque, we might never know that the model was underperforming.

The second dynamic is the lack of strong expectations around the performance of machine learning techniques. Consider how many papers introduce techniques with ["unexpected" performance](https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=%22the+unreasonable+effectiveness+of%22&terms-0-field=all&classification-computer_science=y&classification-physics_archives=all&date-filter_by=all_dates&date-year=&date-from_date=&date-to_date=&date-date_type=submitted_date&abstracts=show&size=50&order=-announced_date_first). We have a case of practice outrunning theory. In general, it is difficult, if not impossible, to estimate the performance a given model will achieve on a given task. For this reason, we are unaware when a model is not living up to its performance potential. Not only can bugs be silent, they can be invisible.

The clearest statement I can make of these ideas is: *"Gradient descent, when applied to models with unclear performance properties, tends to mask bugs in implementation code."*

Okay, I've made my complaint. Now what do we do about this?

One strategy is to strengthen the ties between theoretical specification and practical implementation. This could take the form of libraries to generate human-readable representations of models from code, or even symbolic equations. This output could be compared to model specs or papers to verify correctness.

This isn't a trivial task, especially in complex training setups involving multiple models and dynamic model structures like RNNs. However, incremental progress is possible. Of course, these libraries themselves could be incorrectly implemented, but we decrease the attack surface for bugs with each layer of verification (at the cost of productivity).

Another strategy is increasing the use of toy problems that are simple enough to verify correctness of a model. Synthetic datasets are key here. However, in cases where practical methods frontrun proofs of theoretical performance expectations, it may be difficult to tightly bound the performance of a model architecture or technique using test datasets. 

My last suggestion is less a scientific or programming technique than a societal one: run a bakeoff. If several individuals or teams attempt to implement a model correctly in isolation, there is a chance that each of them can discover issues with one anothers' models. 